<a href="https://colab.research.google.com/github/maximoscortes2/maximoscortes2/blob/main/andrexversor_stam_LSTM1%2BCONV1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow==2.7.0
epocas = 16000
numero = 1 #este define desde donde parte el sampleador
NUM_SAMPLES = 180000 #numero de samples q seran tomados en cuenta para entrenar por cada grabacion
audio_can = 5 #cantidad de archivos a samplear por posicion
ocultos = 56 #tamaño de la red lstm
segundos = 6000 #segundos en milisengundos ... 10000 es 10 segundos
aprendicion = 2.0e-4 #lerning rate
dibuja = False

muestreooo = 44100
VALIDACION = 0.50

if muestreooo == 44100:
  pankito = '44k'
if muestreooo == 48000:
  pankito = '48k'
if muestreooo == 96000:
  pankito = '96k'
if muestreooo == 192000:
  pankito = '192k'
if muestreooo == 384000:
  pankito = '384k'

prefijo = "56ocultos"
lugarsin = 'MIX'

pinpio = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/pinpio'
cochinin = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/cochinon'

directorio = '/content/drive/MyDrive/'+lugarsin+'/'+pankito


Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 665.5 MB 17 kB/s 
     |████████████████████████████████| 463 kB 12.2 MB/s 
     |████████████████████████████████| 1.3 MB 56.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
    Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
      Successfully uninstalled tens

In [3]:
!ls
#!cp "/content/drive/MyDrive/GAIN/384k/pinpio/audio_lsp_9 (2).wav" "/content/drive/MyDrive/REDU/384k/pinpio/" 
#!cp "/content/drive/MyDrive/GAIN/384k/cochinon/audio_lsp_9 (2).wav" "/content/drive/MyDrive/REDU/384k/cochinon/" 
#!sudo rm -R /content/drive/MyDrive/GAIN* 
#!sudo rm -R /content/drive/MyDrive/cochinin*
#!sudo rm -R /content/drive/MyDrive/pinpio*





drive  sample_data


In [4]:
import tensorflow as tf
from tensorflow import keras
import librosa
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from tqdm import tqdm
import os
from pathlib import Path
import random
import sys

from keras.layers import Conv1D, Dense, Dropout, Embedding, LSTM, Bidirectional, MaxPooling1D

import time
import json
from json import JSONEncoder
!pip install CherryPy #webserver package
!pip install cherrypy_cors



# Frequency smoothing
def freqSmooth(x, sm=1.0/24.0):
    s = sm if sm > 1.0 else np.sqrt(2.0**sm)
    N = len(x)
    y = np.zeros_like(x)
    for i in range(N):
        i1 = max(int(np.floor(i/s)), 0)
        i2 = min(int(np.floor(i*s)+1), N-1)
        if i2 > i1:
            y[i] = np.mean(x[i1:i2])
    return y

# FFT with frequency smoothing
def plot_fft(x, fs, sm=1.0/24.0):
    fft = freqSmooth(20 * np.log10(np.abs(np.fft.rfft(x) + 1.0e-9)), sm=sm)
    freqs = np.fft.rfftfreq(len(x), 1.0 / fs)
    return freqs, fft

# load file from FMA dataset
def load_fma_file(files, filepath, fs, N):
    subfolder = files[random.randrange(0, len(files))]
    subfolderpath = filepath + subfolder + '/'
    subfiles = os.listdir(subfolderpath)
    
    mp3path = subfolderpath + subfiles[random.randrange(0, len(subfiles))]
    x, sr = librosa.load(mp3path, sr=fs, mono=True)
        
    start_idx = int(random.uniform(0, len(x) - N))
    return x[start_idx:start_idx+N]

def plot_metric(history, key, val=True):
    loss = history.history[key]
    if val:
        val_loss = history.history['val_' + key]
    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, 'g.', label='Training ' + key)
    if val:
        plt.plot(epochs, val_loss, 'b', label='Validation ' + key)
    plt.title('Training and validation ' + key)
    plt.xlabel('Epochs')
    plt.ylabel(key)
    plt.legend()


def pre_emphasis_filter(x, coeff=0.85):
    return tf.concat([x[:, 0:1, :], x[:, 1:, :] - coeff*x[:, :-1, :]], axis=1)

def dc_loss(target_y, predicted_y):
    return tf.reduce_mean(tf.square(tf.reduce_mean(target_y) - tf.reduce_mean(predicted_y))) / tf.reduce_mean(tf.square(target_y))

# Error-to-signal loss
def esr_loss(target_y, predicted_y, emphasis_func=lambda x : x):
    target_yp = emphasis_func(target_y)
    pred_yp = emphasis_func(predicted_y)
    return tf.reduce_sum(tf.square(target_yp - pred_yp)) / tf.reduce_sum(tf.square(target_yp))

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

class Model():
    def __init__(self, loss_func, optimizer=keras.optimizers.Adam()):
        self.model = keras.Sequential()
        self.opt = optimizer
        self.loss_func = loss_func

        self.train_loss = []
        self.train_err = []
        self.val_loss = []
        self.val_err = []

    def run_training(self, in_train, out_train, epoch_loss, epoch_error, N_skip, N_block):
        N_samples = in_train.shape[1]
        self.model.reset_states() # clear existing state
        self.model(in_train[:, :N_skip, :]) # process some samples to build up state

        # iterate over blocks
        for n in range(N_skip, N_samples-N_block, N_block):
            # compute loss
            with tf.GradientTape() as tape:
                y_pred = self.model(in_train[:, n:n+N_block, :])
                loss = self.loss_func(out_train[:, n:n+N_block, :], y_pred)

            # apply gradients
            grads = tape.gradient(loss, self.model.trainable_variables)
            self.opt.apply_gradients(zip(grads, self.model.trainable_variables))

            # update training metrics
            epoch_loss.update_state(loss)
            epoch_error.update_state(out_train[:, n:n+N_block, :], y_pred)

    def run_validation(self, in_val, out_val, val_loss, val_error):
        self.model.reset_states()
        y_val = self.model(in_val)
        loss = self.loss_func(out_val, y_val)
        val_loss.update_state(loss)
        val_error.update_state(out_val, y_val)

    def run_epoch(self, epoch, in_train, out_train, in_val=None, out_val=None, N_skip=1000, N_block=2048):
        start = time.perf_counter()
        epoch_loss_avg = keras.metrics.Mean()
        epoch_error = keras.metrics.MeanSquaredError()
        val_loss_avg = keras.metrics.Mean()
        val_error = keras.metrics.MeanSquaredError()

        self.run_training(in_train, out_train, epoch_loss_avg, epoch_error, N_skip, N_block)
        if in_val is not None:
            self.run_validation(in_val, out_val, val_loss_avg, val_error)
            
        # end epoch
        self.train_loss.append(epoch_loss_avg.result())
        self.train_err.append(epoch_error.result())
        self.val_loss.append(val_loss_avg.result())
        self.val_err.append(val_error.result())
        dur = time.perf_counter() - start

        print("epocaaa {:03d} - tiempo: {:.4f}s, perdida: {:.6f}, error: {:.6%}, perdida de validacion: {:.6f}, Val_Error: {:.6%}".format(            epoch+1, dur, epoch_loss_avg.result(), epoch_error.result(), val_loss_avg.result(), val_error.result()))

    def train_until(self, loss_stop, in_train, out_train, in_val=None, out_val=None, N_skip=1000, N_block=2048):
        epoch = 0
        while True:
            self.run_epoch(epoch, in_train, out_train, in_val, out_val, N_skip, N_block)

            if self.train_loss[-1] < loss_stop:
                break

            epoch += 1

    def train(self, num_epochs, in_train, out_train, in_val=None, out_val=None, N_skip=1000, N_block=2048, save_model=None, save_hist=None):
        for epoch in range(num_epochs):
            self.run_epoch(epoch, in_train, out_train, in_val, out_val, N_skip, N_block)

            if save_model is not None:
                self.save_model(save_model)
            if save_hist is not None:
                self.save_history(save_hist)
        
        print("redi!")


    def plot_loss(self):
        epochs = range(1, len(self.train_loss) + 1)
        plt.plot(epochs, self.train_loss, 'g.', label='Training Loss')
        plt.plot(epochs, self.val_loss, 'b', label='Validation Loss')
        plt.title('Training and validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

    def plot_error(self):
        epochs = range(1, len(self.train_loss) + 1)
        plt.plot(epochs, self.train_err, 'g.', label='Training Error')
        plt.plot(epochs, self.val_err, 'b', label='Validation Error')
        plt.title('Training and validation Error')
        plt.xlabel('Epochs')
        plt.ylabel('Error')
        plt.legend()

    def save_model_json(self):
        def get_layer_type(layer):
            if isinstance(layer, keras.layers.TimeDistributed):
                return 'time-distributed-dense'

            if isinstance(layer, keras.layers.GRU):
                return 'gru'

            if isinstance(layer, keras.layers.Dense):
                return 'dense'

            if isinstance(layer, keras.layers.Conv1D):
                return 'convolucion'

            if isinstance(layer, keras.layers.LSTM):
                return 'lstm'

            if isinstance(layer, keras.layers.Bidirectional):
                return 'bi'

            return 'unknown'

        def save_layer(layer):
            layercito = {
                "type"    : get_layer_type(layer),
                "shape"   : layer.output_shape,
                "weights" : layer.get_weights()
            }

            return layercito


        model_dict = {}
        model_dict["in_shape"] = self.model.input_shape
        layers = []
        for layer in self.model.layers:
            layersote = save_layer(layer)
            layers.append(layersote)

        model_dict["layers"] = layers
        return model_dict

    def save_model(self, filename):
        model_dict = self.save_model_json()
        with open(filename, 'w') as outfile:
            json.dump(model_dict, outfile, cls=NumpyArrayEncoder)


 

    def load_model_json(self, json,indice):
        in_shape = json["in_shape"][1:]
        self.model.add(keras.layers.InputLayer(input_shape=in_shape))
        with open("headeR.h", "a") as file:
            file.write("\n// Bloque de alturas numero "+str(indice-1)+" q se hace esto \n")
            file.write("// LSP - 2022\n")
            file.write("estructura_lstm1<"+str(ocultos)+",float>  _pos"+str(indice-1)+"={")
            n_layer = 0
            for layer in json["layers"]:
                #print(in_shape)
                #file.write('int dim_layer'+str(n_layer)+'='+str(in_shape)+';\n')
                weights = layer["weights"]
                np_weights = []
                pesitos= str(weights)
                pesotes = pesitos.replace('[','{')
                pesotes = pesotes.replace(']','}')
                pesotes = pesotes.replace('},','}, \n')
                
                pesotes = pesotes[:-1]
                pesotes = pesotes[1:]
                for w in weights:
                    #print(w)
                    #print(pesotes)
                    np_weights.append(np.array(w))


                
                if layer["type"] == 'time-distributed-dense':
                    file.write('\n// pesos de tiempo distribuido denso capa numero '+str(n_layer)+'\n'+str(pesotes)+',')
                    d_layer = keras.layers.Dense(layer["shape"][-1], activation='tanh')
                    d_layer.build(input_shape=in_shape)
                    d_layer.set_weights(np_weights)
                    m_layer = keras.layers.TimeDistributed(d_layer)

                elif layer["type"] == 'gru':
                    file.write('\n// gru numero '+str(n_layer)+'\n'+str(pesotes)+',')
                    m_layer = keras.layers.GRU(units=layer["shape"][-1], return_sequences=True)
                    m_layer.build(input_shape=(None, None, in_shape[-1]))
                    m_layer.set_weights(np_weights)

                elif layer["type"] == 'lstm':
                    file.write('\n// pesos de LSTM capa numero '+str(n_layer)+'\n'+str(pesotes)+',')
                    m_layer = keras.layers.LSTM(units=layer["shape"][-1], activation="tanh", return_sequences=True, recurrent_activation="sigmoid", use_bias=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", bias_initializer="random_normal", unit_forget_bias=False)
                    m_layer.build(input_shape=(None, None, in_shape[-1]))
                    m_layer.set_weights(np_weights)

                elif layer["type"] == 'bi':
                    file.write('\n// pesos de bi-LSTM '+str(n_layer)+'\n'+str(pesotes)+',')
                    m_layer = keras.layers.Bidirectional(LSTM(units=layer["shape"][-1], activation="tanh", return_sequences=True, recurrent_activation="sigmoid", use_bias=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", bias_initializer="random_normal", unit_forget_bias=False), merge_mode='sum' )
                    m_layer.build(input_shape=(None, None, in_shape[-1]))
                    m_layer.set_weights(np_weights)

                elif layer["type"] == 'convolucion':
                    file.write('\n// pesos de convolucion capa numero '+str(n_layer)+'\n'+str(pesotes)+',')
                    m_layer = keras.layers.Conv1D(filters=layer["shape"][-1],kernel_size=9,  activation=None,padding='same',input_shape=(ocultos, 1))#,filters=256, kernel_size=5, padding='same',input_shape=(ocultos, 1)) 
                    m_layer.build(input_shape=(None, None, in_shape[-1]))
                    m_layer.set_weights(np_weights)
                    


                elif layer["type"] == 'dense':
                    file.write('\n// pesos de denso normal numero '+str(n_layer)+'\n'+str(pesotes) +'};')                   
                    m_layer = keras.layers.Dense(layer["shape"][-1])
                    m_layer.build(input_shape=in_shape)
                    m_layer.set_weights(np_weights)

                else:
                    continue
                
                in_shape = tuple(layer["shape"])
                self.model.add(m_layer)
                n_layer = n_layer +1
      
    def load_model(self, filename,indice):
        with open(filename, 'r') as json_file:
            model_json = json.load(json_file)
        self.load_model_json(model_json,indice)

    def save_history(self, filename):
        history = np.array([self.train_loss, self.train_err, self.val_loss, self.val_err])
        np.savetxt(filename, history)

    def load_history(self, filename):
        history = np.loadtxt(filename)
        self.train_loss = history[0].tolist()
        self.train_err = history[1].tolist()
        self.val_loss = history[2].tolist()
        self.val_err = history[3].tolist()




def model_loss(target_y, predicted_y):
    return esr_loss(target_y, predicted_y, pre_emphasis_filter) + dc_loss(target_y, predicted_y)




def rc(a):
    c = 0
    kCharge = 0.5
    kDischarge = 0.5
    rr=a
    ss=0
    for w in a:
        nivel = round(w * 128)
        if nivel > 128:
          nivel = 128
    #		// rectify the signal
        x1 = abs(nivel)
        if x1 > c: 
            c = c * (1 - kCharge) + x1 * kCharge
        else:
            c = c * (1 - kDischarge)
      
        nivel = c;
        rr[ss] = c
        ss=ss+1
    cl = np.asarray(rr)
    return rr

#FRAME_SIZE = 1024
#HOP_LENGTH = 512


FRAME_SIZE = 64
HOP_LENGTH = 1
def amplitude_envelope(signal, frame_size, hop_length):
    """Calculate the amplitude envelope of a signal with a given frame size nad hop length."""
    amplitude_envelope = []
    
    # calculate amplitude envelope for each frame
    #print("envelote", len(signal), " hop:",hop_length)
    for i in range(0, len(signal), hop_length): 
        amplitude_envelope_current_frame = max(signal[i:i+frame_size]) 
        amplitude_envelope.append(amplitude_envelope_current_frame)
    
    return np.array(amplitude_envelope)    
def fancy_amplitude_envelope(signal, frame_size, hop_length):
    """Fancier Python code to calculate the amplitude envelope of a signal with a given frame size."""
    return np.array([max(signal[i:i+frame_size]) for i in range(0, len(signal), hop_length)])

def diplernin(indice):
    mish = str(indice)
    #pathlist = Path(pinpio+mish).glob('*.wav').  para multiproposito
    
    pathlist = Path(pinpio).glob('G99R00CL0*.wav')
    clean_data = []
    dist_data = []
    print(pathlist)
    print(pinpio)

    for p in tqdm(pathlist):
        fs = muestreooo #384000 #np.random.uniform(48000, 96000)
        x, sr = librosa.load(str(p), sr=fs, mono=True)
        if lugarsin == "REDU":
            xx = amplitude_envelope(x, FRAME_SIZE, HOP_LENGTH)
            framesx = range(len(xx))
            tx = librosa.frames_to_time(framesx, hop_length=HOP_LENGTH)
            xx = np.asarray(xx)
        wavedata = os.path.join(".", str(p))
        sampleRate, audioBuffer = scipy.io.wavfile.read(wavedata)
        duration = len(audioBuffer)/sampleRate
        time = np.arange(0,duration,1/sampleRate) #time vector
        if dibuja:
            ax = plt.subplot(3, 1, 1)
            plt.figure(figsize=(300, 20))
            plt.ylim((-1, 1))
            plt.plot(time,x) #audioBuffer)
            plt.xlabel('Time [s]')
            plt.ylabel('Amplitude')
            plt.title(str(p))
            plt.show()     




        #print("muestreo",sr)
        N = len(x)
        #NUM_SAMPLES = len(x)/2
        #for wav_dir in [cochinin+mish+'/']:
        for wav_dir in [cochinin+'/']:
            
            y, sr = librosa.load(wav_dir + p.name, sr=fs, mono=True)

            #print(wav_dir + p.name," y:",y)


            frames = range(len(y))
            t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH)
            #y = rc(y)
            if lugarsin == "REDU":
                yy = amplitude_envelope(y, FRAME_SIZE, HOP_LENGTH)
                frames = range(len(yy))
                t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH)
                yy = np.asarray(yy)

            #clean_data.append(x[N:N+NUM_SAMPLES])
            #dist_data.append(y[N:N+NUM_SAMPLES])

            #clean_data.append(x[N//2:N//2+NUM_SAMPLES])
            #dist_data.append(yy[N//2:N//2+NUM_SAMPLES])
            
            #clean_data.append(x)
            if lugarsin == "MIX":
                #dist_data.append(y)
                clean_data.append(x[N//2:N//2+NUM_SAMPLES])

                dist_data.append(y[N//2:N//2+NUM_SAMPLES])
            if lugarsin == "GAIN":
                #dist_data.append(y)
                clean_data.append(x[N//2:N//2+NUM_SAMPLES])

                dist_data.append(y[N//2:N//2+NUM_SAMPLES])
            if lugarsin == "REDU":

                #clean_data.append(xx[N//2:N//2+NUM_SAMPLES])
                #dist_data.append(yy[N//2:N//2+NUM_SAMPLES])

                clean_data.append(x[N//2:N//2+NUM_SAMPLES])
                dist_data.append(y[N//2:N//2+NUM_SAMPLES])
                #clean_data.append(x)
                #dist_data.append(yy)
                #NUM_SAMPLES = len(x)

                #dist_data.append(yy)
            
            
            

            ppp = str(p.name)
            if dibuja:
                plt.figure(figsize=(300, 20))

                ax = plt.subplot(3, 1, 1)
                #librosa.display.waveplot(y, alpha=0.5)
                if lugarsin == "MIX":
                    plt.plot(t, y, color="m")
                    plt.ylim((-1, 1))
                if lugarsin == "REDU":   
                    #plt.plot(t, yy, color="r")
                    plt.plot(t, xx, color="y")
                    plt.ylim((-1, 1))
                if lugarsin == "GAIN":   
                    plt.plot(t, y, color="r")
                    plt.ylim((-1, 1))
                
                
                plt.title("contorno")



            #wavedata = os.path.join(wav_dir, ppp)

            #print("este es el segundo" + wavedata)
            #sampeRate, audioBuffer = scipy.io.wavfile.read(wavedata)
            #duration = len(audioBuffer)/sampeRate
            #time = np.arange(0,duration,1/sampeRate) #time vector

            #plt.plot(time,y)
            #plt.xlabel('Time [s]')
            #plt.ylabel('Amplitude')
            #plt.title(ppp)
            #plt.show()    


    clean_data = np.asarray(clean_data)
    dist_data = np.asarray(dist_data)
    NUM_FILES = len(clean_data)

    
    
    NUM_TRAIN = int(NUM_FILES * VALIDACION) # validacion es cero coma algo
    NUM_VAL = NUM_FILES - NUM_TRAIN
    NUM_INPUTS = 1


    x_train, x_val = np.split(clean_data, [NUM_TRAIN])  
    y_train, y_val = np.split(dist_data,  [NUM_TRAIN])

    
    OUT_train  = np.reshape(y_train, (NUM_TRAIN, NUM_SAMPLES, NUM_INPUTS))
    OUT_val    = np.reshape(y_val, (NUM_VAL, NUM_SAMPLES, NUM_INPUTS))
    IN_train = np.reshape(x_train, (NUM_TRAIN, NUM_SAMPLES, NUM_INPUTS))
    IN_val   = np.reshape(x_val, (NUM_VAL, NUM_SAMPLES, NUM_INPUTS))


    np.save('/content/drive/MyDrive/'+lugarsin+'/'+pankito+"/info"+mish+"/out_train.npy", OUT_train)
    np.save('/content/drive/MyDrive/'+lugarsin+'/'+pankito+"/info"+mish+"/out_val.npy", OUT_val)
    np.save('/content/drive/MyDrive/'+lugarsin+'/'+pankito+"/info"+mish+"/in_train.npy", IN_train)
    np.save('/content/drive/MyDrive/'+lugarsin+'/'+pankito+"/info"+mish+"/in_val.npy", IN_val)

    OUT_train = np.load('/content/drive/MyDrive/'+lugarsin+'/'+pankito+"/info"+mish+"/out_train.npy")
    OUT_val   = np.load('/content/drive/MyDrive/'+lugarsin+'/'+pankito+"/info"+mish+"/out_val.npy")
    IN_train  = np.load('/content/drive/MyDrive/'+lugarsin+'/'+pankito+"/info"+mish+"/in_train.npy")
    IN_val    = np.load('/content/drive/MyDrive/'+lugarsin+'/'+pankito+"/info"+mish+"/in_val.npy")

    
    NUM_INPUTS = 1
    
    if not os.path.exists('/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos'):
        os.makedirs('/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos')

          
    model_file = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/_elstm-'+prefijo+'_'+pankito+'_'+lugarsin+'_'+mish+'.json'
    model_hist = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/andrex_history'+prefijo+'_'+pankito+'_'+lugarsin+'_'+mish+'.txt'
    
    model = Model(model_loss, optimizer=keras.optimizers.Adam(learning_rate=aprendicion))
    risal = os.path.exists(model_file)
    if risal:
        model.load_model(model_file,indice)
        
    else:
        model.model.add(keras.layers.InputLayer(input_shape=(None, NUM_INPUTS)))
        
        model.model.add(keras.layers.TimeDistributed(keras.layers.Dense(ocultos, activation='tanh')))
       
        model.model.add(Conv1D(filters=ocultos, kernel_size=9, padding='same', activation="sigmoid", input_shape=(ocultos, 1) ))
        model.model.add(keras.layers.LSTM (ocultos, activation="tanh", return_sequences=True, recurrent_activation="sigmoid", use_bias=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", bias_initializer="random_normal", unit_forget_bias=False))
        #activation='relu',
        model.model.add(keras.layers.Dense(1))

    model.model.summary()

    #print("in:",IN_train, "out:",OUT_train, "in:",IN_val, "out:",OUT_val, model_file)
    model.train(epocas, IN_train, OUT_train, IN_val, OUT_val, save_model=model_file, save_hist=model_hist)
    print(len(model.train_loss))
    print(model.train_loss[-1])
    print(model.val_loss[-1])
    for idx in range(1,10):
        predictions = model.model.predict(IN_train[idx].reshape(1, NUM_SAMPLES, NUM_INPUTS)).flatten()
        print(predictions)
        duration = len(predictions)/muestreooo
        time = np.arange(0,duration,1/muestreooo) #time vector
        if dibuja:
            plt.plot(time,predictions)
            plt.figure(figsize=(15, 17))
            plt.xlabel('Time [s]')
            plt.ylabel('Amplitude')
            plt.title(ppp)
            plt.show()    

    print(esr_loss(OUT_val, model.model.predict(IN_val)))
    print(esr_loss(OUT_train, model.model.predict(IN_train)))
    model.save_model(model_file)
    model.save_history(model_hist)
    #files.download('modelos/andrex'+mish+'.json') 
    #files.download('modelos/andrex_history'+mish+'.txt')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 11.6 MB/s 
     |████████████████████████████████| 104 kB 39.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for httpagentparser: filename=httpagentparser-1.9.3-py3-none-any.whl size=7556 sha256=c5595faf84bf9290de7716f9529ea11cfe7eddb7563c5718453559a58c8c53a5
  Stored in directory: /root/.cache/pip/wheels/65/b0/50/1149bd0e56813d74bd594c7a82837763e9d2af33cae86b1d42
Successfully built httpagentparser


In [ ]:
#sampleo y entrenamiento
from os import remove


import cherrypy_cors
from google.colab import files
import cherrypy

import scipy.io.wavfile
from scipy.io import wavfile
from google.colab.output import eval_js
pulentisimo = eval_js("google.colab.kernel.proxyPort(8888)")
print(pulentisimo+'/a')
import subprocess
subprocess.Popen(["python", "/content/test.py", "8888"]) 
f = open("direccion.pan", "w")
f.write(pulentisimo)
f.close()
FINFIN= """ 
<script>location.reload();</script>

"""

import matplotlib.pyplot as plt
import numpy as np
import wave
import sys

numero = 1
while numero < 2:
    mish = str(numero)
    if not os.path.exists(directorio+'/info'+mish):
        os.makedirs(directorio+'/info'+mish)
    on = 0
    while on < audio_can:
        audio = []
        f = open("sinimportancia.chu", "w")
        f.write(str(numero))
        f.close()
        mish = str(numero)
        
        filon = "doble"+str(on)+".wav"
        print("filon : "+ filon)
        
        audion= []
        eee = 0

        on = on + 1

       

    diplernin(numero)
    numero = numero + 1


https://0eum5boveefj-496ff2e9c6d22116-8888-colab.googleusercontent.com//a
filon : doble0.wav
filon : doble1.wav
filon : doble2.wav
filon : doble3.wav
filon : doble4.wav
<generator object Path.glob at 0x7f3c0ba6f450>
/content/drive/MyDrive/MIX/44k/pinpio


50it [00:47,  1.06it/s]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, None, 56)         112       
 ibuted)                                                         
                                                                 
 conv1d (Conv1D)             (None, None, 56)          28280     
                                                                 
 lstm (LSTM)                 (None, None, 56)          25312     
                                                                 
 dense_1 (Dense)             (None, None, 1)           57        
                                                                 
Total params: 53,761
Trainable params: 53,761
Non-trainable params: 0
_________________________________________________________________
epocaaa 001 - tiempo: 24.1605s, perdida: 0.240258, error: 1.176939%, perdida de validacion: 0.171617, Val_Error: 0.319

In [ ]:
#fabricacion de encabezado de c++
import time
numero = 1
ahora = time.strftime("%c")

print( "Fecha y hora " + time.strftime("%c"))

#archivosjason = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/_elstm-'+pankito+'_'+lugarsin+'_'+mish+'.json'
archivosjason = Path('/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos').glob('*.json')
if os.path.exists("headeR.h"):
    os.remove("headeR.h")

with open("headeR.h", "a") as file:
    file.write("// arquitectura4.h es un archivo autogenerado por andrexversor_stam-LSTM1+conv1d\n")
    file.write("// fecha y hora de creacion del archivo !:!::::"+str(ahora)+"\n")
    file.write("// NOMBRE:"+lugarsin+"\n")
    file.write("// muestreo:"+str(muestreooo)+"\n")


    file.write("""#ifndef ARQUITECTURA1_H_INCLUDED
#define ARQUITECTURA1_H_INCLUDED

#include "MotorAndrexDL.h"
#include <assert.h>
#include <limits.h>
#include "math.h"
#include "stdio.h"
#include <algorithm>
#include <JuceHeader.h>

class equipo1
{
public:
	equipo1()
	{
	}
	virtual ~equipo1()
	{
	}
	void inicializar()
	{""")

    file.write("// este al inicializar \n")
    for i in range(1):
        file.write("_reduccion["+str(i)+"] = std::make_unique<modelo_lstm1<"+str(ocultos)+">>(_pos"+str(i)+");\n")
        file.write("_reduccion["+str(i)+"]->reset();\n")
    file.write(""" 		}

	float aplicacion(float dato, int par1, int par2, int par3)
	{ 
		dato = _reduccion[par1]->processSample(dato);
		return dato;
	}

    
    """)
    
    #file.write("std::unique_ptr<modelo_lstm1<"+str(ocultos)+">> _LSTM[100];\n")
    file.write("\n")

model = Model(model_loss, optimizer=keras.optimizers.Adam(learning_rate=aprendicion))
for n in tqdm(archivosjason):
        
        
        for wav_dir in ['/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/']:
            model_file=wav_dir + n.name
            archivitos = model_file
            dd = ''.join([n for n in archivitos if n.isdigit()])
            ddd = int(dd)
           
            print("nombre del modelo:"+model_file)
            print("numero del modelo:"+dd)
            
            model.load_model(model_file,ddd)
           
with open("headeR.h", "a") as file:
    file.write("""
private:


	std::unique_ptr<modelo_lstm1<"""+str(ocultos)+""">> _trafo_IN[3];
	std::unique_ptr<modelo_lstm1<"""+str(ocultos)+""">> _ganancia[3];
	std::unique_ptr<modelo_lstm1<"""+str(ocultos)+""">> _reduccion[100];
	std::unique_ptr<modelo_lstm1<"""+str(ocultos)+""">> _trafo_OUT[3];


};

#endif 
    
    """)        
!cp headeR.h /content/drive/MyDrive/arquitectura4.h

In [ ]:
    # dibujos para comparar
    import scipy.io.wavfile
    from scipy.io import wavfile
    import scipy.signal

    import numpy as numpy
    import pylab as pylab
    import matplotlib.pyplot as plt
    import scipy.fftpack
    import matplotlib.pyplot as plt
    from scipy.fftpack import fft
    from scipy.io import wavfile # get the api
    import matplotlib.pyplot as plt
    from scipy import signal
    from scipy.io import wavfile


    import numpy as np
    from matplotlib import pyplot as plt
    import scipy.io.wavfile as wav
    from numpy.lib import stride_tricks

    import matplotlib.pyplot as plt
   
    import warnings
   
    from scipy.signal import butter, lfilter, freqz
    import matplotlib.pyplot as plt


    def butter_lowpass(cutoff, fs, order=5):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a

    def butter_lowpass_filter(data, cutoff, fs, order=5):
        b, a = butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data)
        return y


    def fftPlot(sig, dt=None, pinta="b", plot=True):
        # Here it's assumes analytic signal (real signal...) - so only half of the axis is required

        if dt is None:
            dt = 1
            t = np.arange(0, sig.shape[-1])
            xLabel = 'samples'
        else:
            t = np.arange(0, sig.shape[-1]) * dt
            xLabel = 'freq [Hz]'

        if sig.shape[0] % 2 != 0:
            warnings.warn("signal preferred to be even in size, autoFixing it...")
            t = t[0:-1]
            sig = sig[0:-1]

        sigFFT = np.fft.fft(sig) / t.shape[0]  # Divided by size t for coherent magnitude

        freq = np.fft.fftfreq(t.shape[0], d=dt)

        # Plot analytic signal - right half of frequence axis needed only...
        firstNegInd = np.argmax(freq < 0)
        freqAxisPos = freq[0:firstNegInd]
        sigFFTPos = 2 * sigFFT[0:firstNegInd]  # *2 because of magnitude of analytic signal

        if plot:
            #plt.figure()
            plt.xscale('log')
            plt.yscale('log')
            plt.plot(freqAxisPos, np.abs(sigFFTPos),color=pinta)
            #plt.xlabel(xLabel)
            #plt.ylabel('mag')
            #plt.title('Analytic FFT plot')
            #plt.show()

        return sigFFTPos, freqAxisPos


    """ short time fourier transform of audio signal """
    def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):
        win = window(frameSize)
        hopSize = int(frameSize - np.floor(overlapFac * frameSize))

        # zeros at beginning (thus center of 1st window should be for sample nr. 0)   
        samples = np.append(np.zeros(int(np.floor(frameSize/2.0))), sig)    
        # cols for windowing
        cols = np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1
        # zeros at end (thus samples can be fully covered by frames)
        samples = np.append(samples, np.zeros(frameSize))

        frames = stride_tricks.as_strided(samples, shape=(int(cols), frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
        frames *= win

        return np.fft.rfft(frames)    

    """ scale frequency axis logarithmically """    
    def logscale_spec(spec, sr=44100, factor=20.):
        timebins, freqbins = np.shape(spec)

        scale = np.linspace(0, 1, freqbins) ** factor
        scale *= (freqbins-1)/max(scale)
        scale = np.unique(np.round(scale))

        # create spectrogram with new freq bins
        newspec = np.complex128(np.zeros([timebins, len(scale)]))
        for i in range(0, len(scale)):        
            if i == len(scale)-1:
                newspec[:,i] = np.sum(spec[:,int(scale[i]):], axis=1)
            else:        
                newspec[:,i] = np.sum(spec[:,int(scale[i]):int(scale[i+1])], axis=1)

        # list center freq of bins
        allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
        freqs = []
        for i in range(0, len(scale)):
            if i == len(scale)-1:
                freqs += [np.mean(allfreqs[int(scale[i]):])]
            else:
                freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]

        return newspec, freqs

    """ plot spectrogram"""
    def plotstft(samplerate, samples, binsize=2**10, plotpath=None, colormap="jet"):
        #samplerate, samples = wav.read(audiopath)

        s = stft(samples, binsize)

        sshow, freq = logscale_spec(s, factor=1.0, sr=samplerate)

        ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel

        timebins, freqbins = np.shape(ims)


        print("timebins: ", timebins)
        print("freqbins: ", freqbins)

        plt.figure(figsize=(15, 7.5))
        
       
        plt.imshow(np.transpose(ims), origin="lower", aspect="auto", cmap=colormap, interpolation="none")
        plt.clim([-40 ,60])
        plt.colorbar(fraction=0.03, pad=0.04)
        plt.xlabel("time (s)")
        plt.ylabel("frequency (hz)")
        plt.xlim([0, timebins-1])
        plt.ylim([0, freqbins])

        xlocs = np.float32(np.linspace(0, timebins-1, 5))
        plt.xticks(xlocs, ["%.02f" % l for l in ((xlocs*len(samples)/timebins)+(0.5*binsize))/samplerate])
        ylocs = np.int16(np.round(np.linspace(0, freqbins-1, 10)))
        plt.yticks(ylocs, ["%.02f" % freq[i] for i in ylocs])

        if plotpath:
            plt.savefig(plotpath, bbox_inches="tight")
        else:
            plt.show()

        plt.clf()

        return ims

    


    idx = -1
    indice = 1
    mish = str(indice)
    #model_file = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/andrex'+mish+'.json'
    #model_hist = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/andrex_history'+mish+'.txt'
    model_file = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/_elstm-'+prefijo+'_'+pankito+'_'+lugarsin+'_'+mish+'.json'
    model_hist = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/andrex_history'+prefijo+'_'+pankito+'_'+lugarsin+'_'+mish+'.txt'
    
    model = Model(model_loss, optimizer=keras.optimizers.Adam(learning_rate=aprendicion))
    model.load_model(model_file,1)
    model.model.summary()
    #pathlist = Path(pinpio+mish).glob('*.wav').  para multiproposito
    pathlist = Path(pinpio).glob('*.wav')
    clean_data = []
    dist_data = []
    #print(pathlist)
    #print(pinpio)
    for p in tqdm(pathlist):
        fs = muestreooo #384000 #np.random.uniform(48000, 96000)
        x, sr = librosa.load(str(p), sr=fs, mono=True)
        print(sr)
        wavedata = os.path.join(".", str(p))
        sampleRate, audioBuffer = scipy.io.wavfile.read(wavedata)
        duration = len(audioBuffer)/sampleRate
        time = np.arange(0,duration,1/sampleRate) #time vector
        t = librosa.frames_to_time
        N = len(x)
        #for wav_dir in [cochinin+mish+'/']:
        for wav_dir in [cochinin+'/']:
            
            y, sr = librosa.load(wav_dir + p.name, sr=fs, mono=True)

            #print(wav_dir + p.name," y:",y)


            frames = range(len(y))
            t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH)
            #y = rc(y)
            if lugarsin == "REDU":

                yy = amplitude_envelope(y, FRAME_SIZE, HOP_LENGTH)

                xx = amplitude_envelope(x, FRAME_SIZE, HOP_LENGTH)
                frames = range(len(yy))
                t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH)
                yy = np.asarray(yy)
                xx = np.asarray(xx)

            
            clean_data.append(x)
            NUM_SAMPLES = len(x)
            if lugarsin == "GAIN":
                dist_data.append(y)
            if lugarsin == "MIX":
                dist_data.append(y)
            if lugarsin == "REDU":
                dist_data.append(yy)
            
            
            
            idx = idx + 1
            ppp = str(p.name)
            

            print(clean_data)
            print(idx)
            predictions = model.model.predict(clean_data[idx].reshape(1, NUM_SAMPLES, 1)).flatten()
            print(predictions)
            duration = len(predictions)/muestreooo
            time = np.arange(1,duration,1/muestreooo) #time vector}
            #time = t
            #plt.ylim((0, 1))
            plt.figure(figsize=(300, 5))
            #plt.plot(t,x) #audioBuffer)
            if lugarsin == "REDU":   
                #print(time,yy)
                FRAME_SIZE = 1024
                HOP_LENGTH = 1
                #y =  amplitude_envelope(y, FRAME_SIZE, HOP_LENGTH)  
                #y =  np.asarray(y) 
                plt.plot(t, yy, color="r")
                pasabajo = butter_lowpass_filter(predictions, 0.3, fs, order=5)
                xxx = amplitude_envelope(predictions, 64, HOP_LENGTH)
                xxx = amplitude_envelope(xxx, 16, HOP_LENGTH)

                plt.plot(t,xxx, color="m")

                predictions5 = scipy.signal.savgol_filter(xxx,12333,3) # window size 51, polynomial order 3
                plt.plot(t,predictions5, color="g")
                predictions9 = scipy.signal.savgol_filter(xxx, 26333,4) # window size 51, polynomial order 3
                plt.plot(t,predictions9, color="y")

            if lugarsin == "GAIN":   
                plt.plot(t, y, color="r")
                plt.plot(t,predictions, color="y")
            if lugarsin == "MIX":   
                plt.plot(t,x,color="b")
                plt.plot(t, y, color="r")
                plt.plot(t,predictions, color="y")
                
          

            # plt.xlabel('Time [s]')
            #plt.ylabel('Amplitude')
            #plt.title("prediccion")
            plt.show() 

            zoom = 3000
            plt.figure(figsize=(300, 5))
            plt.plot(time[:zoom],x[:zoom]) #audioBuffer)
            if lugarsin == "REDU":   
                plt.plot(time[:zoom], yy[:zoom], color="r")
            if lugarsin == "GAIN":   
                plt.plot(time[:zoom], y[:zoom], color="r")
            if lugarsin == "MIX":   
                plt.plot(time[:zoom], y[:zoom], color="r")
            plt.plot(time[:zoom],predictions[:zoom], color="g")
            #ims3 = plotstft(sr,predictions)
            #fftPlot(predictions)
            plt.show()
            plt.figure(figsize=(300, 5))
            fftPlot(x,40000, "b")
             
                #frames = range(len(yy))
                #t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH)
                #yy = np.asarray(yy)
            fftPlot(y,40000, "r")
            fftPlot(predictions,40000,"g")
            plt.show()

    


    

In [ ]:
    #prediccion y creacion de archivo
    fs=44100
    from scipy.io.wavfile import write
    idx = -1
    indice = 1
    mish = str(indice)
    #model_file = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/andrex'+mish+'.json'
    model_file = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/_elstm-'+prefijo+'_'+pankito+'_'+lugarsin+'_'+mish+'.json'
    
    model_hist = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/modelos/andrex_history'+mish+'.txt'
    model = Model(model_loss, optimizer=keras.optimizers.Adam(learning_rate=aprendicion))
    model.load_model(model_file,1)
    model.model.summary()
    j, sr = librosa.load("87_30db.wav", sr=fs, mono=True)
    NUM_SAMPLES = len(j)
    predictions = model.model.predict(j.reshape(1, NUM_SAMPLES, 1)).flatten()
    write("example.wav", sr, predictions.astype(np.float32))
    #librosa.output.write_wav('file_trim_5s.wav', predictions, sr)
    print("listoco")    

In [ ]:
    #conversion de 384khz al resto de frecuencias
    !pip install scikits.samplerate
    !apt-get -qq install -y libsamplerate0-dev
    !pip install -qU samplerate scipy resampy torchaudio julius soxr nnresample
    !pip install -qU git+https://github.com/gregorias/samplerate.git
    !pip install -qU git+https://github.com/danpovey/filtering.git
    import samplerate  # https://pypi.org/project/samplerate/
    #import scikits.samplerate  # https://github.com/gregorias/samplerate
    import scipy.signal
    import resampy
    #import scikits.samplerate 
    from scipy.io.wavfile import write
    for panky in (44100,48000,96000,192000):
        if panky == 44100:
          pankoto = '44k'
        if panky == 48000:
          pankoto = '48k'
        if panky == 96000:
          pankoto = '96k'
        if panky == 192000:
          pankoto = '192k'

        P = muestreooo
        Q = panky
        VMIN = -150
        ccc=0
        pinpio = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/pinpio'
        cochinin = '/content/drive/MyDrive/'+lugarsin+'/'+pankito+'/cochinon'
        pinpio2 = '/content/drive/MyDrive/'+lugarsin+'/'+pankoto+'/pinpio'
        cochinin2 = '/content/drive/MyDrive/'+lugarsin+'/'+pankoto+'/cochinon'
        if not os.path.exists(pinpio2):
            os.makedirs(pinpio2)
        if not os.path.exists(cochinin2):
            os.makedirs(cochinin2)
        print("cochinin2: "+cochinin)
        lista_de_wavs = Path(pinpio).glob('*.wav')
        print(lista_de_wavs)
        for p in tqdm(lista_de_wavs):
            fs = muestreooo #384000 #np.random.uniform(48000, 96000)
            x, sr = librosa.load(str(p), sr=fs, mono=True)
            print(sr)
            print(str(p))
            wavedata = os.path.join(".", str(p))
            print(wavedata)
            nuevo = samplerate.resample(x, Q/P, 'sinc_best')
            plt.figure(figsize=(15,3))
            plt.specgram(nuevo,
                        scale='dB', Fs=Q, vmin=VMIN, cmap='inferno')
            plt.colorbar()
            _=plt.axis((0,2,0,Q/2))
    
            write(pinpio2+"/"+str(p.name), panky, nuevo.astype(np.float32))
            #ccc=ccc+1

            N = len(x)
            #for wav_dir in [cochinin+mish+'/']:
            for wav_dir in [cochinin+'/']:
                
                y, sr = librosa.load(wav_dir + p.name, sr=fs, mono=True)
                nuevo = samplerate.resample(y, Q/P, 'sinc_best')
                plt.figure(figsize=(15,3))
                plt.specgram(nuevo,
                            scale='dB', Fs=Q, vmin=VMIN, cmap='inferno')
                plt.colorbar()
                _=plt.axis((0,2,0,Q/2))
        
                write(cochinin2+"/"+str(p.name), panky, nuevo.astype(np.float32))
                ccc=ccc+1

                #print(wav_dir + p.name," y:",y)


                frames = range(len(y))
                t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH)
                #y = rc(y)


                

                
                
            
            #idx = idx + 1
            #ppp = str(p.name)
     
